* Import necessary libraries.

In [1]:
import pandas as pd

from datetime import datetime

* Read clean dataset.

In [2]:
books_df = pd.read_csv('cleaned_books.csv')

* Format date in the df.

In [3]:
books_df['publication_date'] = pd.to_datetime(books_df['publication_date'], format='%m/%d/%Y')

* Prepare DimBook data.

In [4]:
dim_book = books_df[['bookID', 'title', 'num_pages', 'isbn', 'isbn13']].drop_duplicates(subset='bookID')

* Prepare DimAuthor data.

In [5]:
dim_author = books_df[['author_id', 'authors']].drop_duplicates()
dim_author.rename(columns={'authors': 'author'}, inplace=True)

* Prepare DimLanguage data.

In [6]:
dim_language = books_df[['language_id', 'language_code']].drop_duplicates()

* Prepare DimPublisher data.

In [7]:
dim_publisher = books_df[['publisher_id', 'publisher']].drop_duplicates()

* Prepare DimTime data.

In [8]:
# Set date range
start_date = '1900-01-01'
end_date = '2020-03-31'
date_range = pd.date_range(start=start_date, end=end_date)

# Create a DataFrame with calculated time attributes
dim_time = pd.DataFrame({
    'time_id': range(1, len(date_range) + 1),  # Primary Key
    'date': date_range,
    'year': date_range.year,
    'quarter': date_range.quarter,
    'month': date_range.month,
    'day': date_range.day,
    'week': date_range.isocalendar().week,
    'day_of_week': date_range.dayofweek + 1,  # Monday=0 in Python, adjust to SQL
    'is_weekend': date_range.dayofweek.isin([5, 6]).astype(int),  # Saturday, Sunday
})

* Prepare DimBookAuthor data.

In [9]:
dim_book_author = books_df[['bookID', 'author_id']].drop_duplicates()

* Prepare DimBookPublisher data.

In [10]:
dim_book_publisher = books_df[['bookID', 'publisher_id']].drop_duplicates()

* Prepare FactBook data.

In [11]:
fact_book = books_df[['bookID', 'author_id', 'language_id', 'publisher_id', 
                      'average_rating', 'ratings_count', 'text_reviews_count']].copy()

# Map publication_date to time_id in the fact table
date_to_time_id = dict(zip(dim_time['date'], dim_time['time_id']))
fact_book['time_id'] = books_df['publication_date'].map(date_to_time_id)

# Remove duplicate rows in fact_book (one per book, author, and publisher)
fact_book = fact_book.drop_duplicates(subset=['bookID', 'author_id', 'publisher_id'])

* Export dataframes to CSV files.

In [12]:
#dim_book.to_csv('dim_book.csv', index=False)
#dim_author.to_csv('dim_author.csv', index=False)
#dim_language.to_csv('dim_language.csv', index=False)
#dim_publisher.to_csv('dim_publisher.csv', index=False)
#dim_time.to_csv('dim_time.csv', index=False)
#dim_book_author.to_csv('dim_book_author.csv', index=False)
#dim_book_publisher.to_csv('dim_book_publisher.csv', index=False)
#fact_book.to_csv('fact_book.csv', index=True)